Serverino per immagini da unreal

In [3]:
import numpy as np
import socket
from IPython.display import clear_output
import threading
import cv2


HOST = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 4502  # Port to listen on (non-privileged ports are > 1023)
PACKET_SIZE = 1048576

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

facce = 0

contatorePacchetti = 0
contatorePacchettiVecchio = 0
pacchettiPerSecondo = 0

#riconoscimento facciale
def adj_detect_face(img):
    
    face_img = img.copy()
  
    face_rects = face_cascade.detectMultiScale(face_img,scaleFactor=1.2, minNeighbors=5) 
    
    for (x,y,w,h) in face_rects: 
        cv2.rectangle(face_img, (x,y), (x+w,y+h), (255,255,255), 10) 
        
    return (face_img, len(face_rects))

#contatore di pacchetti
def contatore():
    clear_output(wait=True)
    global contatorePacchettiVecchio
    global pacchettiPerSecondo
    #print("Pacchetti al secondo:")
    #print(contatorePacchetti-contatorePacchettiVecchio)
    pacchettiPerSecondo = contatorePacchetti-contatorePacchettiVecchio
    contatorePacchettiVecchio = contatorePacchetti
    t = threading.Timer(1, contatore)
    t.start()
 
#### decommentare per avere il contatore
contatore()

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            
            data = conn.recv(PACKET_SIZE)
            
            #verifico che ci sia un pacchetto da gestire
            if not data:
                break

            #aumento contatore pacchetti
            contatorePacchetti+=1
            
            if ( len(data) == PACKET_SIZE ):
                clear_output(wait=True)
                print(f"Pacchetti per secondo: {pacchettiPerSecondo}")

                #gestisco immagine
                immagine = np.frombuffer(data,dtype=np.uint8)
                immagine = np.reshape(immagine,(512,512,4))
                immagine,facce = adj_detect_face(immagine)
                print(f"Facce rilevate: {bytes(facce)}")

                #resize se necessario
                #immagine = cv2.resize(immagine, (128,256), interpolation = cv2.INTER_AREA)

                cv2.imshow("Feed Video",immagine)
                
            conn.sendall(bytes([facce]))

            #distruggo tutto se premo Q
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

Pacchetti per secondo: 3
Facce rilevate: b'\x00'
